## 호텔 예약 수요
### 포르투갈 호텔 예약 수요 데이터세트
  - 도심 호텔 및 리조트 호텔의 예약 정보
    - 예약 시기, 숙박 기간, 성인, 어린이 및 유아 수, 주차 가능 공간 수 등

In [59]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px

In [60]:
df = pd.read_csv('/home/leejieun/workspace/kaggle-notebooks/datasets/jessemostipak/hotel-booking-demand/hotel_bookings.csv')

In [86]:
df['hotel'].unique()
df['customer_type'].unique()
df.groupby('required_car_parking_spaces').count()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,total_of_special_requests,reservation_status,reservation_status_date
required_car_parking_spaces,,,,,,,,,,,,,,,,,,,,,
0,111974,111974,111974,111974,111974,111974,111974,111974,111974,111974,...,111974,111974,97592,6037,111974,111974,111974,111974,111974,111974
1,7383,7383,7383,7383,7383,7383,7383,7383,7383,7383,...,7383,7383,5435,756,7383,7383,7383,7383,7383,7383
2,28,28,28,28,28,28,28,28,28,28,...,28,28,20,4,28,28,28,28,28,28
3,3,3,3,3,3,3,3,3,3,3,...,3,3,3,0,3,3,3,3,3,3
8,2,2,2,2,2,2,2,2,2,2,...,2,2,0,0,2,2,2,2,2,2


In [80]:
print(df.shape)
print(df['is_canceled'].value_counts())
print(df.columns)
df.head()

(119390, 32)
is_canceled
0    75166
1    44224
Name: count, dtype: int64
Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date'],
      dtype='object')


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [62]:
df.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [63]:
df.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

## null 값 추출하여 drop

In [64]:
nan_replacements = {'children': 0.0, 'country':'Unknown', 'agent':0, 'company':0}
df_cln = df.fillna(nan_replacements)
df_cln['meal'].replace('Undefined', "SC", inplace=True)

zero_guests = list(df_cln.loc[df_cln['adults']
                              + df_cln['children']
                              + df_cln['babies'] ==0].index)
df_cln.drop(df_cln.index[zero_guests], inplace=True)

/tmp/ipykernel_1985586/3310363977.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cln['meal'].replace('Undefined', "SC", inplace=True)


In [65]:
df_cln.shape

(119210, 32)

In [66]:
df.shape

(119390, 32)

In [67]:
rh = df_cln.loc[(df_cln['hotel'] == 'Resort Hotel') & (df_cln['is_canceled'] ==0)]
ch = df_cln.loc[(df_cln['hotel'] == 'City Hotel') & (df_cln['is_canceled'] ==0)]

In [68]:
df_cln['country'].value_counts()

country
PRT    48483
GBR    12120
FRA    10401
ESP     8560
DEU     7285
       ...  
DJI        1
BWA        1
HND        1
VGB        1
NAM        1
Name: count, Length: 178, dtype: int64

## 국가별 실제 손님 수

In [87]:
country_data = pd.DataFrame(df_cln.loc[df_cln["is_canceled"] == 0]["country"].value_counts())
country_data.reset_index(level=0, inplace=True)
country_data[country_data["country"] =='KOR']

,country,count
37,KOR,78


In [75]:
country_data.rename(columns={"country": "Number of Guests"}, inplace=True)
country_data

,Number of Guests,count
0,PRT,20977
1,GBR,9668
2,FRA,8468
3,ESP,6383
4,DEU,6067
...,...,...
161,BHR,1
162,DJI,1
163,MLI,1
164,NPL,1


In [92]:
total_guests = country_data["count"].sum()
country_data["Guests in %"] = round(country_data["count"] / total_guests * 100, 2)
country_data["country"] = country_data.index
country_data.loc[country_data["Guests in %"] < 2, 'country'] = 'Other'

fig = px.pie(country_data, values="count",
             names='country',
             title='Home country of guests',
             template='seaborn'
             )
fig.update_traces(textposition='inside', textinfo='value+percent+label')
fig.show()

/tmp/ipykernel_1985586/78594080.py:4: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Other' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

